In [4]:
import numpy as np
import scipy.linalg

# CTMC

Continuous-time stochastic process $\{X(t),t\ge 0\}$ on a finite state space with Markov property at each time $t\ge 0$. Finite-state CTMC spends an exponentially distributed amount of time in a given state before jumping out of it. Thus *exponential distributions* play an important role in CTMCs. 


#### Exponential distribution
Recall that for an exponential random variable $T\sim Exp(\lambda)$:
- $E[T] = \frac{1}{\lambda}$ and
- $Var[T] = \frac{1}{\lambda^2}$.

**Exercise**: Suppose a new machine is put into operation at
time zero. Its lifetime is known to be an $Exp(\lambda)$ random variable with $\lambda = 0.1(1/hour)$.
What is the probability that the machine will give trouble-free service continuously for 1 day? Suppose the machine has not failed by the end of the first day. What is the probability that it will give trouble-free service for the whole of the next day?


#### The minimum of independent exponential random variables
Let $T_i$ be an $Exp(\lambda_i)$ random variable $(1\le i\le k)$, we can think of $T_i$ as the time when an event of type $i$ occurs and suppose $T_1, T_2,\dots , T_k$ are
independent. The r.v. $T = \min\{T_1, T_2,\dots , T_k\}$, representing the
time when the first of these k events occurs, is an exponential r.v. with $\lambda = \sum_{i=1}^k \lambda_i$.

**Exercise**: A hospital currently has
seven pregnant women waiting to give birth. Three of them are expected to
give birth to boys, and the remaining four are expected to give birth to girls.
From prior experience, the hospital staff knows that a mother spends on average 6 hours in the hospital before delivering a boy and 5 hours before delivering
a girl. Assume that these times are independent and exponentially distributed.
What is the probability that the first baby born is a boy and is born in the next
hour?

#### CTMC
Let $X(t)$ be the state of a system at time t. 

Random evolution of the system: suppose the system starts in state i. It stays there for an $Exp(q_i)$ amount of time, called the *sojourn time in state i*. At the end of the sojourn time in
state i, the system makes a sudden transition to state $j\ne i$ with probability $\pi_{i,j}$,
independent of how long the system has been in state i. In case state i is absorbing we set $q_i=0$.

Analogously to DTMCs, a CTMC can also be represented graphically by means
of a directed graph: the directed graph has one node for
each state. There is a directed arc from node i to node j if $\pi_{i,j} > 0$. The quantity
$q_{i,j}= q_i \pi_{i,j}$, called the transition rate from i to j, is written next to this arc. Note that there are no self-loops. Note that we can recover the original parameters $q_i$ and $\pi_{i,j}$ from the rates $q_{i,j}$:
$$q_i = \sum_{j}q_{i,j}$$
$$\pi_{i,j} = \frac{q_{i,j}}{q_i}.$$

In each state $i$, there's a race condition between $k$ edges, each exponentially distributed with rate $q_{ij}$, the exit rate is $q_{i,i} = -q_i$. The matrix $Q = (q_{ij})_{ij}$ is called **infinitesiaml generator** of the CTMC.
Each row sum up to 0.


Define a class CTMC, with a method `__init__()` to initialize the infinitesiamal generator

In [47]:
class CTMC:
    
    def __set_init_prob(self, distr, n):
        if distr == None: 
            return np.repeat(1/n, n)
        else:
            return distr
    
    def __create_Q(self, inf_gen):
        Q = inf_gen
        # setta la diagonale di Q
        for i in range(self.size):
            Q[i][i] = np.sum([Q[i][j] for j in range(self.size)])
        
        if(self.not_well_defined(Q)):
            raise Exception("The input matrix is not well defined")
        else:    
            return Q

    def __init__(self, inf_gen, distr=None):
        self.init_prob_distr = self.__set_init_prob(distr, len(inf_gen))
        self.size = len(inf_gen)
        self.Q = self.__create_Q(inf_gen)
        
    def not_well_defined(self, inf_gen):
        return any(inf_gen[i][i]<0 for i in range(self.size))
        

Consider the SIR model and define a function `infinitesimal_generator_SIR()`that takes the rates $q_S, q_I, q_R$ as inputs and returns the infinitesimal generator `Q` after checking that it is well-defined.

In [53]:
def infinitesimal_generator_SIR(qs, qi, qr):
    return CTMC([[0, qs, 0],
                 [0, 0, qr],
                 [qi, 0, 0]]).Q

Q_SIR = infinitesimal_generator_SIR(qs=0.3, qi=0.5, qr=0.2)
Q_SIR

[[0.3, 0.3, 0], [0, 0.2, 0.2], [0.5, 0, 0.5]]

#### Jump chain and holding times

We can factorize a CTMC $X(t)$ in a DTMC $Y_n$ called **jump chain**, with probability matrix $\Pi$ where $\pi_{i,j}=\frac{q_{i,j}}{-q_{i,i}}$ if $i\ne j$ and $\pi_{ii}=0$ .

Define a function `jump_chain_matrix()` which takes the infinitesimal generator `Q` as input and returns the transition matrix $\Pi$.


In [81]:
def jump_chain_matrix(Q):
    Pi = np.repeat(np.zeros(len(Q)),len(Q))
    for i in range(len(Q)):
        for j in range(len(Q)):
            if j!=i:
                if Q[i][i] != 0:
                    print( - Q[i][j] / Q[i][i])
                    Pi[i][j] = - Q[i][j] / Q[i][i]
                else:
                    Pi[i][j] == 1
    return Pi

In [82]:
jump_chain_matrix(Q_SIR)

-1.0


TypeError: 'numpy.float64' object does not support item assignment

Recall the Kolmogorov equation and define a function `continuous_trajectories()` taking as input $Q$, the initial distribution $p_0$ and a time $t$. Use a solver for differential equations and return the solution. 

Consider one of the models define before and plot the trajectory of each state against time.

### Poisson process 

Consider systems whose state transitions are triggered by streams of
events that occur one at a time and assume that the successive inter-event
times are iid exponential random variables.
$N_{\lambda}(0,t)$ is a process that counts how many times an exponential distribution with rate $\lambda$ has fired from time $0$ to time $t$. It can be seen as a CTMC with $S=\mathbb{N}$ and $Q$ is: $q_{i,i+1}=\lambda$ and zero elsewhere. The time $t_i = t_{i-1}+D_i$, with $D_i\sim Exp(\lambda)$.



**Exercise**: A machine shop consists of N machines
and M repair persons ($M\le N$). The machines are identical, and the lifetimes
of the machines are independent $Exp(\mu)$ random variables. When the machines fail,
they are serviced in the order of failure by the M repair persons. Each failed machine
needs one and only one repair person, and the repair times are independent $Exp(\lambda)$
random variables. A repaired machine behaves like a new machine. Let $X(t)$ be the
number of machines that are functioning at time t.

Draw the diagram for $N = 4$ and $M = 2$ and define a function that takes $N, M, \lambda$ and $\mu$ as input and return the infitesimal generator.

**Exercise**: A commercial jet airplane has four engines, two on each wing. Each engine lasts for a random amount of time that is an exponential random variable with parameter $\lambda$ and then fails. If the failure takes place in
flight, there can be no repair. The airplane needs at least one engine on each wing to
function properly in order to fly safely. Model this system so that we can predict the
probability of a trouble-free flight.

Hint: Draw the diagram first.

#### Birth-death process and queues

General class of CTMC on $S = \mathbb{N}$ with birth rate $\lambda_i$ (from $i$ to $i+1$) and death rate $\mu_i$ (from $i$ to $i-1$). When a birth occurs, the process goes from state $i$ to $i + 1$. When a death occurs, the process goes from state $i$ to state $i − 1$.

The birth–death process is the most fundamental example of a **queueing model**. This is a queue with Poisson arrivals, drawn from an infinite population, and $C$ servers with exponentially distributed service time with $K$ places in the queue. 


Define a general `BirthDeath` class inheriting from the class CTMC in order to initialize the number of states, the rates of forward and backward transitions.

**Exercise**: A telephone switch can handle K calls at any
one time. Calls arrive according to a Poisson process with rate $\lambda$. If the switch is
already serving K calls when a new call arrives, then the new call is lost. If a call
is accepted, it lasts for an $Exp(\mu)$ amount of time and then terminates. All call
durations are independent of each other. Let $X(t)$ be the number of calls that are
being handled by the switch at time t. Model $X(t)$ as a CTMC.

What if the caller can put a maximum of H callers on hold?

**Exercise**: (Inventory Management). A retail store manages the inventory of
washing machines as follows. When the number of washing machines in stock decreases to a fixed number k, an order is placed with the manufacturer for `m` new
washing machines. It takes a random amount of time for the order to be delivered
to the retailer. If the inventory is at most `k` when an order is delivered (including the
newly delivered order), another order for m items is placed immediately. Suppose
the delivery times are iid $Exp(\lambda)$ and that the demand for the washing machines occurs according to a $PP(\mu)$. Demands that cannot be immediately satisfied are lost.
Model this system as a CTMC.

#### Thinning algorithm (time-inhomogeneous scenario)
1. Take $\lambda^* = \max_t \lambda(t)$,
2. generate arrival times $t_0^*,t_1^*,\dots $ of a stationary Poisson process with $\lambda^*$.
3. reject $t_i^*$ with probability $1-\frac{\lambda(t_i^*)}{\lambda^*}= 1- p(t_i^*)$.